# Исследование надежности заемщиков.

# Описание.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Шаг 1. Обзор данных

### Шаг 1.1. Импорт библиотек.

In [1]:
import pandas as pd 
import numpy as np
data = pd.read_csv('/datasets/data.csv')


### Шаг 1.2. Изучение данных

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,1.935100e+04,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,1.674223e+05,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,1.029716e+05,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.066726e+04,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.030532e+05,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.450179e+05,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,2.034351e+05,NaN


Заметим, что в столбце days_employed and children and dob_years содержатся ненормальные для них значения

### Шаг 2.1 Заполнение пропусков

In [4]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data[data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Заметим, что там, где пропуск в days_employed, у того же объекта пропуск и в total_income. Пропусков 2174, что достаточно много для того что бы не обращать на них внимания и удалить их. Поэтому логично будет сгруппировать по полу и возрасту, а затем для каждой группы найти медиану и присвоить ее пропускам из той же группы

Создадим сводную таблицу для анализа пропусков:

In [6]:
pivot_table_days_employed = data.pivot_table(index=['dob_years'],
                                columns = 'gender',
                                values = 'days_employed',
                                aggfunc='median')
pivot_table_days_employed

gender,F,M,XNA
dob_years,,,
0,-1327.579211,-1015.395451,NaN
19,-710.230781,-885.268574,NaN
20,-595.559207,-880.528819,NaN
21,-608.003610,-627.185770,NaN
22,-691.825293,-740.851560,NaN
23,-634.611807,-727.699529,NaN
24,-964.894524,-901.549138,-2358.600502
25,-951.181756,-869.061319,NaN
26,-1173.584347,-957.617546,NaN


In [7]:
#Создаем переменную, в которой будем хранить значение медианы для стоблца days_employed
days_employed_median = data['days_employed'].median()
#Подсчет общего кол-ва записей
days_employed_count = data['days_employed'].count()
#Подсчет доли 
days_employed_share = days_employed_median / days_employed_count

In [8]:

#Замена пропусков
def fillna_in_days_employed(data_frame):
    gender = data_frame['gender']
    age = data_frame['dob_years']
    if pd.isnull(data_frame['days_employed']):
        return pivot_table_days_employed.loc[age,gender]
    else:
        return data_frame['days_employed']

data['days_employed'] = data.apply(fillna_in_days_employed, axis = 1)



In [9]:
print('Количество пропусков:', data['days_employed'].isnull().sum())
print()
print('Значение медианы для дней работы:', days_employed_median)
print()
print('Всего записей -', days_employed_count)
print()
print(days_employed_share, '- доля пропущенных значений')

Количество пропусков: 0

Значение медианы для дней работы: -1203.369528770489

Всего записей - 19351

-0.062186425960957525 - доля пропущенных значений


In [10]:
pivot_table_total_income = data.pivot_table(index=['dob_years'],
                                    columns = 'gender',
                                    values = 'total_income',
                                    aggfunc='median')
pivot_table_total_income

gender,F,M,XNA
dob_years,,,
0,117279.001914,179138.212343,NaN
19,112217.756662,91876.754772,NaN
20,101512.989536,134750.749515,NaN
21,112778.853259,147812.788085,NaN
22,111633.034748,158258.854591,NaN
23,117755.877467,137544.158776,NaN
24,122193.154166,153115.076039,203905.157261
25,131764.441672,161883.762815,NaN
26,132408.440852,165998.158511,NaN


In [11]:
#Создаем переменную, в которой будем хранить значение медианы для стоблца total_income
total_income_median = data['total_income'].median()
#Подсчет общего кол-ва записей
total_income_count = data['total_income'].count()
#Подсчет доли 
total_income_share = total_income_median / total_income_count

In [12]:
#Замена пропусков
def fillna_in_total_income(data_frame):
    gender = data_frame['gender']
    age = data_frame['dob_years']
    if pd.isnull(data_frame['total_income']):
        return pivot_table_total_income.loc[age,gender]
    else:
        return data_frame['total_income']

data['total_income'] = data.apply(fillna_in_total_income, axis = 1)

In [13]:
#Проверка 
print('Количество пропусков:', data['total_income'].isnull().sum())
print()
print('Значение медианы для общей прибыли:', total_income_median) 
print()
print('Всего записей -', total_income_count)
print()
print(total_income_share, '- доля пропущенных значений')

Количество пропусков: 0

Значение медианы для общей прибыли: 145017.93753253992

Всего записей - 19351

7.494079765001287 - доля пропущенных значений


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Тк обработать огромное количество данных сложно и тем более ответить на вопрос, какой доход и стаж работы нормальные, те взять среднее значение, поэтому логичнее взять медиану для заполнения пропусков

### Шаг 2.2 Проверка данных на аномалии и исправления.

### Шаг 2.2.1 Проверим, есть ли другие отрицательные значения в столбце children

In [15]:
for column in data.columns:
    try:
        data[data[column] < 0]
    except:
        print('не числовой столбец')

не числовой столбец
не числовой столбец
не числовой столбец
не числовой столбец
не числовой столбец


Чтобы выявить анамалии просмотрим каждый столбец по отдельности 

In [16]:
data[data['children'] < 0]['children'].unique()


array([-1])

In [17]:
data[data['children'] > 10]['children'].unique()

array([20])

Другие значения присутствуют. Скорее всего это техническая ошибка, которую следует исправить заменой.

In [18]:
data['children'].replace(-1,1,inplace = True)

In [19]:
data['children'].replace(20,2,inplace = True)

Заметим, что все анаомальные занчения были изменины, код работает верно

### Шаг 2.2.2 Проверим столбец days_employed

На 2022 пенсионный возраст женщин 56, мужчин 61. Заметим, что человек проживший 100 лет и работающий с самого своего рождения может максимально иметь ~36500 дней рабочего стажа(это тоже сомнительно), в столбце ответственном за хранение стажа преимущественно отрицательные значения или значение превосходящие человеческие возможности. Проверим есть ли в данном столбце значения большие 0, но при этом не превосходящие человеческие возможности(36500 дней стажа)

In [20]:
data[(data['days_employed'] >= 0) & (data['days_employed'] < 36500)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Заметим, что корректные данные отсутсвуют. Скорее всего это произошло из-за технической ошибки (корректные данные приняли отрицательные значения). Отсюда можно придти к решению:
Откинуть минус и считать данные, которые являются отрицательными истинными, но содержавшими аномалию в виде минуса. А значения не укладывающиеся в человеческие возможности привести к максимальному стажу.

In [56]:
#В данных встречается аномалия: отрицательное значение
data['days_employed'] = data.days_employed.abs()
print(data['days_employed'])

0         8437.673028
1         4024.803754
2         5623.422610
3         4124.747207
4        14235.000000
             ...     
21466     4529.316663
21467    15330.000000
21468     2113.346888
21469     3112.481705
21470     1984.507589
Name: days_employed, Length: 21471, dtype: float64


### Напишем функцию, которая вычисляет максимальный рабочий стаж

In [22]:
def max_seniority(data_frame):
    
    current_seniority = data_frame['days_employed']
    
    if current_seniority == np.nan:
        return current_seniority
    else:
        if data_frame['gender'] == 'M' and data_frame['dob_years'] > 61 : 
            max_years = 61
        elif data_frame['gender'] == 'F' and data_frame['dob_years'] > 56:
            max_years = 56
        else: 
            max_years = data_frame['dob_years']

        max_seniority = (max_years - 14) * 365
        if current_seniority > max_seniority:
            return max_seniority
        elif current_seniority < 0:
            return (current_seniority * -1)
        else:
            return current_seniority

In [23]:
data['days_employed'] = data.apply(max_seniority, axis = 1)

Проверим изменения

In [24]:
data['days_employed']<0

0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Name: days_employed, Length: 21525, dtype: bool

In [25]:
data['days_employed']>35600

0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Name: days_employed, Length: 21525, dtype: bool

Вывод

1)Мы поняли, что в данных присутствовали аномалии

2)Таблица нуждается в очистке и исправления

### Шаг 2.3. Изменение типов данных.

Перед тем, как изменять тип данных, проверим наличие пустых значений и заменим их на медиану столбца

In [27]:
print(data['total_income'].isnull().sum()) # для проверки наличия пропусков

0


In [28]:
data['total_income'] = data['total_income'].astype('int') #изменение типа данных

### Шаг 2.4. Удаление дубликатов.

In [29]:
data = data.drop_duplicates().reset_index(drop = True)

In [30]:
data.duplicated().sum()

0

In [31]:
#Проверим на неявные дубликаты
print(data['education'].value_counts())

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [32]:
#Переведем названия моделей в нижний регистр методом str.lower()
data['education'] = data['education'].str.lower() 
print(data['education'].value_counts()) 

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [33]:
#Проверим на неявные дубликаты
print(data['family_status'].value_counts())

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [34]:
#Переведем названия моделей в нижний регистр методом str.lower()
data['family_status'] = data['family_status'].str.lower()
print(data['family_status'].value_counts())

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [35]:
print(data['gender'].value_counts())

F      14189
M       7281
XNA        1
Name: gender, dtype: int64


In [36]:
print(data['income_type'].value_counts())

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


In [37]:
print(data['purpose'].value_counts())

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [38]:
data_education = data[['education','education_id']]
data_education

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21466,среднее,1
21467,среднее,1
21468,среднее,1
21469,среднее,1


In [39]:
data_family_status = data[['family_status','family_status_id']]
data_family_status

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
...,...,...
21466,гражданский брак,1
21467,женат / замужем,0
21468,гражданский брак,1
21469,женат / замужем,0


In [40]:
data.drop(columns=['family_status', 'education'],axis = 1,inplace = True)

### Шаг 2.6. Категоризация дохода.

In [41]:
print(data['total_income'])

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21466    224791
21467    155999
21468     89672
21469    244093
21470     82047
Name: total_income, Length: 21471, dtype: int64


In [42]:
def total_income_category(messages):
    if messages >=0 and messages <= 30000:
        return 'E'
    if messages >= 30001 and messages <= 50000:
        return 'D'
    if messages >= 50001 and messages <= 200000:
        return 'C'
    if messages >=200001 and messages <= 1000000:
        return 'B'
    return 'A'

data['total_income_category'] = data['total_income'].apply(total_income_category)


### Шаг 2.7. Категоризация целей кредита.

In [43]:
def purpose_category(purpose):
    if 'авто' in purpose  :
        return 'операции с автомобилем'
    if 'недви' in purpose:
        return 'операции с недвижимостью'
    if 'жил' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    return 'получение образования'

data['purpose_category'] = data['purpose'].apply(purpose_category)
print(data['purpose_category'])

0        операции с недвижимостью
1          операции с автомобилем
2        операции с недвижимостью
3           получение образования
4              проведение свадьбы
                   ...           
21466    операции с недвижимостью
21467      операции с автомобилем
21468    операции с недвижимостью
21469      операции с автомобилем
21470      операции с автомобилем
Name: purpose_category, Length: 21471, dtype: object


Жилье и недвижимость это грубо говоря одно и тоже, а ремонт и строительство это достаточно близкие понятие.

In [44]:
#код ревьюера
data['purpose_category'].value_counts()

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

Катетеризируем количество детей разделим и на две группы "семьи без детей" и "cемьи с детьми". Напишем для этого функцию

In [45]:
def children_category(child):
    if child == 0  :
        return 'семьи без детей'
    return 'cемьи с детьми'

data['children_category'] = data['children'].apply(children_category)
print(data['children_category'])

0         cемьи с детьми
1         cемьи с детьми
2        семьи без детей
3         cемьи с детьми
4        семьи без детей
              ...       
21466     cемьи с детьми
21467    семьи без детей
21468     cемьи с детьми
21469     cемьи с детьми
21470     cемьи с детьми
Name: children_category, Length: 21471, dtype: object


### Вывод

Теперь основываясь на категории мы сможем изучить влияет ли какая-то категория на вероятность задолженности по кредиту

### Ответы на вопросы.

##### Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [46]:
data_of_family = data.groupby(['children_category','debt']).agg({'debt':["count"]})
data_of_family

debt
                        count
children_category debt       
cемьи с детьми    0      6686
                  1       678
семьи без детей   0     13044
                  1      1063

Заметим, что вероятность клиента принадлежащего к группе "семьи с детьми" стать должником составляет 9.2%, а вероятность клиента принадлежащего к группе "семьи без детей" стать должником составляет 7.5%

##### Вывод 1:

Да, зависимость есть: вероятность задолжности у семьи с детьми на 1,7% выше, чем у семей без детей

##### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
data_of_family_status = data.groupby(['family_status_id','debt']).agg({'debt':["count"]})
data_of_family_status

debt
                       count
family_status_id debt       
0                0     11413
                 1       931
1                0      3775
                 1       388
2                0       896
                 1        63
3                0      1110
                 1        85
4                0      2536
                 1       274

Вероятность клиента принадлежащего к группе "в разводе" стать должником составляет 7.1%

Вероятность клиента принадлежащего к группе "вдовец / вдова" стать должником составляет 6.6%

Вероятность клиента принадлежащего к группе "гражданский брак" стать должником составляет 9.3%

Вероятность клиента принадлежащего к группе "женат / замужем" стать должником составляет 7.5%

Вероятность клиента принадлежащего к группе "не женат / не замужем" стать должником составляет 9.8%


##### Вывод 2:

Не состоящие в браке наиболее склонны к задержке выплат, в то же время вдовцы/вдвы меньше всех склонны к задолженностям по кредиту

##### Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [48]:
data_of_income_status = data.groupby(['total_income_category','debt']).agg({'debt':["count"]})
data_of_income_status

debt
                            count
total_income_category debt       
A                     0        23
                      1         2
B                     0      4685
                      1       356
C                     0     14673
                      1      1360
D                     0       329
                      1        21
E                     0        20
                      1         2

Вероятность клиента принадлежащего к группе "A" стать должником составляет 8.0%

Вероятность клиента принадлежащего к группе "B" стать должником составляет 7.06%

Вероятность клиента принадлежащего к группе "C" стать должником составляет 6.0%

Вероятность клиента принадлежащего к группе "E" стать должником составляет 9.09%

##### Вывод 3:

Наибольшая вероятность 9.09% стать должником есть у людей,
принадлежащего к группе "E"

##### Вопрос 4:
Как разные цели кредита влияют на его возврат в срок?

In [49]:
data_of_purpose_type = data.groupby(['purpose_category','debt']).agg({'debt':["count"]})
data_of_purpose_type

debt
                               count
purpose_category         debt       
операции с автомобилем   0      3905
                         1       403
операции с недвижимостью 0     10032
                         1       782
получение образования    0      3644
                         1       370
проведение свадьбы       0      2149
                         1       186

Вероятность клиента принадлежащего к группе "операции с автомобилем" стать должником составляет 9.35%

Вероятность клиента принадлежащего к группе "операции с недвижимостью" стать должником составляет 7.46%

Вероятность клиента принадлежащего к группе "получение образования" стать должником составляет 8.0%

Вероятность клиента принадлежащего к группе "проведение свадьбы" стать должником составляет 8.0%

##### Вывод 4:

Заметим, что наибольшая вероятность стать должником, у людей, берущих кредит на "операции с автомобилем". 

При этом вероятность задолжности у людей принадлежащего к группе "получение образования" и принадлежащего к группе "проведение свадьбы" равна - это скорее всего происходит из-за того, что возрастная категория людей приблизительно равна и варьруется от 20-35 лет.

In [50]:
data_pivot_category = data[['debt','purpose_category']]
data_pivot_category

,debt,purpose_category
0,0,операции с недвижимостью
1,0,операции с автомобилем
2,0,операции с недвижимостью
3,0,получение образования
4,0,проведение свадьбы
...,...,...
21466,0,операции с недвижимостью
21467,0,операции с автомобилем
21468,1,операции с недвижимостью
21469,1,операции с автомобилем


In [51]:
table_category = pd.pivot_table(data_pivot_category,
                                index=['purpose_category'],
                                values=['debt'],
                                aggfunc='mean',
                                fill_value='NaN'     
)
table_category

,debt
purpose_category,
операции с автомобилем,0.093547
операции с недвижимостью,0.072314
получение образования,0.092177
проведение свадьбы,0.079657


## Общий вывод:

Основываясь на полученные данные можно прийти к следующему выводу: Семейное положение и наличие детей в семье безусловно влияют на факт погашения кредита. Так мы можем увидеть, что вероятность клиента принадлежащего к группе "семьи с детьми" стать должником составляет 9.2%, а вероятность клиента принадлежащего к группе "не женат / не замужем" стать должником составляет 9.8%. Поэтому логично будет предположить, что наиболее безопасными для банка заемщиками являются семьи без детей и люди являющиеся вдовой/вдовцом

In [52]:
data_pivot = data[['children','days_employed','dob_years',
                 'education_id','family_status_id','gender',
                 'income_type','total_income_category',
                  'purpose_category']]
data_pivot

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,B,получение образования
4,0,14235.000000,53,1,1,F,пенсионер,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,B,операции с недвижимостью
21467,0,15330.000000,67,1,0,F,пенсионер,C,операции с автомобилем
21468,1,2113.346888,38,1,1,M,сотрудник,C,операции с недвижимостью
21469,3,3112.481705,38,1,0,M,сотрудник,B,операции с автомобилем


In [53]:
table = pd.pivot_table(data_pivot,
                       index=['education_id','total_income_category','family_status_id','purpose_category'],
                       values=['days_employed','dob_years','children','income_type'],
                       aggfunc='median',
                       fill_value=0)
table

children  \
education_id total_income_category family_status_id purpose_category                     
0            A                     0                операции с автомобилем         1.0   
                                                    операции с недвижимостью       1.0   
                                                    получение образования          2.0   
                                   1                операции с недвижимостью       0.5   
                                                    проведение свадьбы             0.5   
...                                                                                ...   
4            B                     0                операции с недвижимостью       0.0   
                                                    получение образования          0.0   
             C                     0                операции с недвижимостью       1.5   
                                   3                операции с недвижимостью       1.0   
                                   4                операции с недвижимостью       0.0   

                                                                              days_employed  \
education_id total_income_category family_status_id purpose_category                          
0            A                     0                операции с автомобилем      1283.436610   
                                                    операции с недвижимостью    2577.664662   
                                                    получение образования       1654.855987   
                                   1                операции с недвижимостью    3694.000583   
                                                    проведение свадьбы          3034.031302   
...                                                                                     ...   
4            B                     0                операции с недвижимостью   15330.000000   
                                                    получение образования       5352.038180   
             C                     0                операции с недвижимостью   11561.537942   
                                   3                операции с недвижимостью    2351.431934   
                                   4                операции с недвижимостью     409.200149   

                                                                              dob_years  
education_id total_income_category family_status_id purpose_category                     
0            A                     0                операции с автомобилем         53.0  
                                                    операции с недвижимостью       44.0  
                                                    получение образования          39.0  
                                   1                операции с недвижимостью       35.5  
                                                    проведение свадьбы             41.5  
...                                                                                 ...  
4            B                     0                операции с недвижимостью       62.0  
                                                    получение образования          58.0  
             C                     0                операции с недвижимостью       52.5  
                                   3                операции с недвижимостью       37.0  
                                   4                операции с недвижимостью       45.0  

[181 rows x 3 columns]